In [ ]:
import requests
import pandas as pd
from zipfile import ZipFile 
import io
import json

# Download the EPC Schema

The EPC schema is only included in .zip files, so we make an empty request to the API with a .zip format to extract the schema file and save it.

In [ ]:
with open('epc_token.txt', 'r') as file:
    epc_token = file.read()

schema_headers = {
    'Accept': 'application/zip',
    'Authorization': f'Basic {epc_token}'
}

schema_params = {
    'size': 0,
}

url = 'https://epc.opendatacommunities.org/api/v1/domestic/search'

In [ ]:
response = requests.get(url, headers=schema_headers, params=schema_params)
response.status_code

In [ ]:
zip = ZipFile(io.BytesIO(response.content))
schema_path = zip.extract(member="schema.json", path="EPC")

In [ ]:
with open (schema_path, 'r') as file:
    schema = json.load(file)

# The first API request

Make a small API request with json format

In [ ]:
with open('epc_token.txt', 'r') as file:
    epc_token = file.read()

headers = {
    'Accept': 'application/json',
    'Authorization': f'Basic {epc_token}'
}

params = {
    'size': 100,
    'postcode': 'M1'
}

url = 'https://epc.opendatacommunities.org/api/v1/domestic/search'

In [ ]:
response = requests.get(url, headers=headers, params=params)
response.status_code
data = response.json()

# Download all data using pagination

In [ ]:
with open('epc_token.txt', 'r') as file:
    epc_token = file.read()

headers = {
    'Accept': 'application/json',
    'Authorization': f'Basic {epc_token}'
}

params = {
    'postcode': 'M1',
    'size': 5000
}

url = 'https://epc.opendatacommunities.org/api/v1/domestic/search'

In [ ]:
response = requests.get(url, headers=headers, params=params)
response.status_code
data = response.json()
search_after = response.headers['X-Next-Search-After']

In [ ]:
while search_after != None:
    params["search-after"] = search_after
    response = requests.get(url, headers=headers, params=params)
    page_data = response.json()
    data['rows'] = data['rows'] + page_data['rows']
    try:
        search_after = response.headers['X-Next-Search-After']
    except KeyError:
        search_after = None

# PART 2: Data Transformation

**Our question: for each type of property, for each tenure, what is the average increase in energy efficiency as a percentage of the current energy efficiency?**

## Converting the data

From the last article, we have our raw data in a dictionary called data . The column names are defined in a list at the index 'column-names' while the actual data is in a list at index 'rows' . We will use both of these lists to create a Pandas DataFrame epc_df 

In [ ]:
epc_df = pd.DataFrame(columns=data['column-names'], data=data['rows'])

## Understanding the Data

DataFrames have lots of inbuilt attributes and methods to help you understand the data. 

*Note: when using a Jupyter notebook, you do not need to use `print` statements; the output of the last line of each cell is printed below the cell.*

### Understanding the data as a whole
* Identify what data is present
* Identify where the data we need is

In [ ]:
# the shape of the DataFrame (no. rows, no. columns)
epc_df.shape

In [ ]:
# the column names
epc_df.columns

In [ ]:
# the first 15 rows of the data
epc_df.head(15)

### Understanding the content and quality of the data
* Identify the data cleaning tasks necessary by inspecting the columns we will use

In [ ]:
# the data type of the column
epc_df['potential-energy-efficiency'].dtype

In [ ]:
# the data type of the column
epc_df['tenure'].dtype

In [ ]:
# the number of NA (blank) values in the column
sum(epc_df['tenure'].isna())

In [ ]:
# the smallest value in the column
epc_df['potential-energy-efficiency'].min()

In [ ]:
# the largest value in the column
epc_df['potential-energy-efficiency'].max()

In [ ]:
# the counts for each unique value in the column
epc_df['tenure'].value_counts()

In [ ]:
# the counts for each unique value in the column
epc_df['property-type'].value_counts()

In [ ]:
# the length of the list of unique values in the column
len(epc_df['uprn'].unique())

**Findings:**

- The datatype returned for each column is `dtype('O')` , meaning ‘Object’, instead of the proper [Pandas datatype](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes) to represent the data.
- We can see from the `tenure` column value counts that there are inconsistencies in categorising the same meaning with text.
- We can also see that there are some entries that mean ‘No data’ in this context; such as ‘Not defined - ..’.
- We can see that there are fewer unique values in the `uprn` column than there are rows in the table, meaning there are some duplicate entires for each property.

## Cleaning the data
* Clean the data according to the data quality needs understood previously

### Convert datatypes:
We can convert most datatypes using the `astype` function. To parse the inspection-date information into a datetime object we use the pandas `to_datetime` function. This allows us to specify the format the date is written in.

In [ ]:
epc_df['inspection-date'] = pd.to_datetime(epc_df['inspection-date'], format="%Y-%m-%d")
epc_df['current-energy-efficiency'] = epc_df['current-energy-efficiency'].astype(int)
epc_df['potential-energy-efficiency'] = epc_df['potential-energy-efficiency'].astype(int)
epc_df['property-type'] = epc_df['property-type'].astype(str)
epc_df['tenure'] = epc_df['tenure'].astype(str)

### Convert categorical variables

We can map the unwanted variations on the categorical values using the `replace` function. Here we provide the function with a dictionary with the unwanted values as keys and the wanted values as values.

In [ ]:
epc_df['tenure'] = epc_df['tenure'].replace({
    "Rented (private)":"rental (private)",
    "Rented (social)":"rental (social)",
    "Owner-occupied":"owner-occupied",    
})

In [ ]:
# check the results of replacing
epc_df['tenure'].value_counts()

### Clear 'no data' cells
We select all the rows in the column that do not contain the three categories we have kept, and set them all to be `None` .

In [ ]:
epc_df['tenure'] = epc_df['tenure'].where(epc_df['tenure'].isin(["rental (private)","rental (social)","owner-occupied" ]), None)

### Remove duplicates

To keep only the newest entry for duplicate UPRNs, we first sort by `inspection-date` in descending order. Then we drop duplicates, looking only at the column `UPRN` to identify duplicates, and keep the first value found.

In [ ]:
epc_df = epc_df.sort_values(by='inspection-date', ascending=False)

In [ ]:
epc_df = epc_df.drop_duplicates(subset=["uprn"], keep='first')

In [ ]:
epc_df = epc_df[["tenure","property-type","current-energy-efficiency","potential-energy-efficiency"]]
epc_df = epc_df.dropna()

## Manipulating the Data
* Perform the tasks to extract the required information to answer our question:

_**Our question: for each type of property, for each tenure, what is the average increase in energy efficiency as a percentage of the current energy efficiency?**_

### Calculate the potential efficiency increase

We can use the mathematical operators on DataFrame columns, and the operators are applied row by row. Assigning the result to a new column name creates a new column in the DataFrame

In [ ]:
epc_df["potential-efficiency-increase"] = (epc_df['potential-energy-efficiency'] - epc_df['current-energy-efficiency']) / epc_df['current-energy-efficiency']

### Group and calculate averages
The `groupby` function creates a grouped DataFrame according to the unique values in the columns provided. We can then use the built in `mean` function to calculate the average potential efficiency across each group. Finally, the `unstack` function ungroups the DataFrame so it can be displayed as a flat table.

In [ ]:
epc_df_group = epc_df.groupby(["tenure","property-type"])

In [ ]:
efficiency_increase_averages = epc_df_group["potential-efficiency-increase"].mean().unstack()

In [ ]:
efficiency_increase_averages

### Save the result
Pandas has functions to save DataFrames to a variety of file formats. Here we use `to_csv` to save it to a .csv format.

In [ ]:
efficiency_increase_averages.to_csv("averages.csv")